In [ ]:
import gymnasium as gym
import highway_env
import time

print("--- Starting VISUAL test ---")
print("A new window should pop up. This is NOT training.")

# 1. Create the environment AND tell it to show the window
# THIS IS THE KEY LINE: render_mode='human'
env = gym.make('highway-v0', render_mode='human')

# 2. Reset the environment (starts the simulation)
obs, info = env.reset()

# 3. Run the simulation for 200 steps so you can watch
for _ in range(200):
    
    # 4. Choose a RANDOM action (just to make it move)
    action = env.action_space.sample() 
    
    # 5. Take the action
    obs, reward, done, truncated, info = env.step(action)
    
    # 6. If the car crashes or the time ends, reset it
    if done or truncated:
        print("Episode finished. Resetting...")
        obs, info = env.reset()

# 7. Close the window when the loop is done
env.close()
print("--- Visual test complete. ---")

/home/jmffelisberto/Desktop/Modelling-Simulation/venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Starting training...
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 16       |
|    ep_rew_mean      | 13.4     |
|    exploration_rate | 0.97     |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2        |
|    time_elapsed     | 22       |
|    total_timesteps  | 64       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 15.2     |
|    ep_rew_mean      | 12.1     |
|    exploration_rate | 0.942    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2        |
|    time_elapsed     | 41       |
|    total_timesteps  | 122      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.449    |
|    n_updates        | 5        |
---------